In [3]:
import pandas as pd
import sys
import os
import json
import re
from tqdm import tqdm
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import time


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, pairwise
import io
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from openai import OpenAI
from typing import Literal, TypedDict, Literal, Dict, Any, List, Union

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser


In [4]:
with io.open('/Users/emanueleferrari/desktop/Key.txt', 'r', encoding='utf-8') as file:
        api_key = file.read().strip()

os.environ['OPENAI_API_KEY'] = api_key
del api_key

client = OpenAI()


In [5]:
# SYSTEM PROMPT
extract_toc_system = """You are an assistans specialized extracting item headings and subheadings from Table of Contents (TOC) of SEC 10-K filings.

Your task is to reason about the input and extract headings and subheadings and their corresponding page numbers.
You must also label with the corresponding item label, or with 'subheading'.

**Here is a reference of the item labels and their typical headings:**

    item01: item 1. business,
    item01and02: item 1&2. business and properties,
    item01a: item 1a. risk factors,
    item01b: item 1b. unresolved staff comments,
    item01c: item 1c. cybersecurity,
    item02: item 2. properties,
    item03: item 3. legal proceedings,
    item04: item 4. mine safety disclosures,
    item05: item 5. market for registrant’s common equity, related stockholder matters and issuer purchases of equity securities,
    item06: item 6. selected financial data,
    item07: item 7. management’s discussion and analysis of financial condition and results of operations,
    item07a: item 7a. quantitative and qualitative disclosures about market risk,
    item08: item 8. financial statements and supplementary data,
    item09: item 9. changes in and disagreements with accountants on accounting and financial disclosure,
    item09a: item 9a. controls and procedures,
    item09b: item 9b. other information,
    item09c: item 9c. disclosure regarding foreign jurisdictions that prevent inspections,
    item10: item 10. directors, executive officers and corporate governance,
    item11: item 11. executive compensation,
    item12: item 12. security ownership of certain beneficial owners and management and related stockholder matters,
    item13: item 13. certain relationships and related transactions, and director independence,
    item14: item 14. principal accountant fees and services,
    item15: item 15. exhibits, financial statement schedules,
    item16: item 16. form 10-k summary

**Some important consideration about item headings:**
Sometimes item01 and item02 appear combined as item01and02.
There must be only one item and relative label per entry, and the order should be sequential (e.g., item01, item01a item02, item03...) always controll the page number.
Items item01c, item09b, item09c, and item16 are optional and will be missing most of the time, the others should be present.
A company may be exempt from reporting certain items or have none, typically item03, item04, or item06. In such cases, the item heading can be replaced with labels like "Item 3. None," "Item 6. Reserved," "Item 4. Not Applicable," "Item 3. Omitted," or similar.
Items may have minor variations, such as different uses of singular and plural forms or inclusion of the company's name, but they should be easily recognizable at the correct target.
Items headings are usually divided in foru parts: Part I, Part II, Part III, and Part IV. You can ignore the part headings and focus only on the item headings and their subheadings.

**Other subheadings**
The table of content may contain also other subheadings, you should report also them. The label for all subheading is 'subheading'.

**Additional information**
The table of content may contain also some narrative text, like a foward looking statement or other clarification statements.
You should ignore it and focus only on the headings and subheadings.

**IF YOU ARE NOT ABLE TO EXTRACT THE TABLE OF CONTENTS, YOU CAN REPORT IT IN YOUR REASONING AND RETURN AN EMPTY LIST OF TOC ELEMENTS**
"""

examples = """
**Examples:**
Input:

On March 28, 2015, the aggregate market value of the registrant’s Class A Common Stock, $0.10 par value (Class A stock), and Class B Common Stock, $0.10 par value (Class B stock), held
by non-affiliates of the registrant was $11,395,283,906 and $412,319, respectively. Class B stock is not publicly listed for trade on any exchange or market system. However, Class B stock is
convertible into Class A stock on a share-for-share basis, so the market value was calculated based on the market price of Class A stock.
On October 31, 2015, there were 295,644,459 shares of Class A stock and 70,010,805 shares of Class B stock outstanding.
**INCORPORATION BY REFERENCE**
Portions of the registrant’s definitive Proxy Statement for the registrant’s Annual Meeting of Shareholders to be held February 5, 2016, are incorporated by reference into Part III of this
Annual Report on Form 10-K.
**TABLE OF CONTENTS**
PAGE
PART I
Item 1. Business 2
Item 1A. Risk Factors 6
Item 1B. Unresolved Staff Comments 14
Item 2. Properties 15
Item 3. Legal Proceedings 16
Item 4. Mine Safety Disclosures 16
PART II
Item 5. Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities 19
Item 6. Selected Financial Data 21
Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations 23
Item 7A. Quantitative and Qualitative Disclosures About Market Risk 43
Item 8. Financial Statements and Supplementary Data 45
Item 9. Changes in and Disagreements With Accountants on Accounting and Financial Disclosure 93
Item 9A. Controls and Procedures 93
Item 9B. Other Information 93
PART III
Item 10. Directors, Executive Officers and Corporate Governance 94
Item 11. Executive Compensation 94
Item 12. Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters 94
Item 13. Certain Relationships and Related Transactions, and Director Independence 94
Item 14. Principal Accounting Fees and Services 94
PART IV
Item 15. Exhibits, Financial Statement Schedules 95
1
 
output:
reasoning: The table of content is well structured in four parts, with the corresponding page numbers at the end of each line corresponding. There are no subheadings. The text also contain some  narrative text at the top, I will ignore it.
toc_elements:[
    {
    text: "Item 1. Business",
    label: "item01",
    page: "2"
    },
    {
    text: "Item 1A. Risk Factors",
    label: "item01a",
    page: "6"
    },
    {
    text: "Item 1B. Unresolved Staff Comments",
    label: "item01b",
    page: "14"
    },
    {
    text: "Item 2. Properties",
    label: "item02",
    page: "15"
    },
    {
    text: "Item 3. Legal Proceedings",
    label: "item03",
    page: "16"
    },
    {
    text: "Item 4. Mine Safety Disclosures",
    label: "item04",
    page: "16"
    },
    {
    text: "Item 5. Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities",
    label: "item05",
    page: "19"
    },
    {
    text: "Item 6. Selected Financial Data",
    label: "item06",
    page: "21"
    },
    {
    text: "Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations",
    label: "item07",
    page: "23"
    },
    {
    text: "Item 7A. Quantitative and Qualitative Disclosures About Market Risk",
    label: "item07a",
    page: "43"
    },
    {
    text: "Item 8. Financial Statements and Supplementary Data",
    label: "item08",
    page: "45"
    },
    {
    text: "Item 9. Changes in and Disagreements With Accountants on Accounting and Financial Disclosure",
    label: "item09",
    page: "93"
    },
    {
    text: "Item 9A. Controls and Procedures",
    label: "item09a",
    page: "93"
    },
    {
    text: "Item 9B. Other Information",
    label: "item09b",
    page: "93"
    },
    {
    text: "Item 10. Directors, Executive Officers and Corporate Governance",
    label: "item10",
    page: "94"
    },
    {
    text: "Item 11. Executive Compensation",
    label: "item11",
    page: "94"
    },
    {
    text: "Item 12. Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters",
    label: "item12",
    page: "94"
    },
    {
    text: "Item 13. Certain Relationships and Related Transactions, and Director Independence",
    label: "item13",
    page: "94"
    },
    {
    text: "Item 14. Principal Accounting Fees and Services",
    label: "item14",
    page: "94"
    },
    {
    text: "Item 15. Exhibits, Financial Statement Schedules",
    label: "item15",
    page: "95"
    }
]

Input:
**Part II**
Item 5 Market for Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities 30
Item 6 Selected Financial Data 31
Item 7 Management’s Discussion and Analysis of Financial Condition and Results of Operations 32
Overview 32
Results of Operations 40
Automotive Sector 42
Financial Services Sector 55
Liquidity and Capital Resources 61
2013 Planning Assumptions and Key Metrics 73
Production Volumes 73
Outlook 74
Critical Accounting Estimates 78
Accounting Standards Issued But Not Yet Adopted 82
Aggregate Contractual Obligations 83
Item 7A Quantitative and Qualitative Disclosures About Market Risk 84

output:
**FORD MOTOR COMPANY**
**ANNUAL REPORT ON FORM 10-K**
**For the Year Ended December 31, 2013**
**Table of Contents** **Page**
**Part I**
Item 1 Business 1
Overview 2
Automotive Sector 3
Financial Services Sector 10
Governmental Standards 11
Employment Data 16
Engineering, Research, and Development 16
Item 1A Risk Factors 17
Item 1B Unresolved Staff Comments 23
Item 2 Properties 24
Item 3 Legal Proceedings 26
Item 4 Mine Safety Disclosures 28
Item 4A Executive Officers of Ford 29
**Part II**
Item 5 Market for Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities 30
Item 6 Selected Financial Data 31
Item 7 Management’s Discussion and Analysis of Financial Condition and Results of Operations 32
Overview 32
Results of Operations 40
Automotive Sector 42
Financial Services Sector 55
Liquidity and Capital Resources 61
2013 Planning Assumptions and Key Metrics 73
Production Volumes 73
Outlook 74
Critical Accounting Estimates 78
Accounting Standards Issued But Not Yet Adopted 82
Aggregate Contractual Obligations 83
Item 7A Quantitative and Qualitative Disclosures About Market Risk 84
Overview 84
Automotive Sector 84
Financial Services Sector 86
Item 8 Financial Statements and Supplementary Data 87
Item 9 Changes in and Disagreements with Accountants on Accounting and Financial Disclosure 87
Item 9A Controls and Procedures 88
Item 9B Other Information 88
**Part III**
Item 10 Directors, Executive Officers of Ford, and Corporate Governance 89
Item 11 Executive Compensation 89
Item 12 Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters 89
Item 13 Certain Relationships and Related Transactions, and Director Independence 89
Item 14 Principal Accounting Fees and Services 89
i
**Table of Contents**
**(continued)**
**Part IV**
Item 15 Exhibits and Financial Statement Schedules 90
Signatures 94
**Ford Motor Company and Subsidiaries Financial Statements**
Report of Independent Registered Public Accounting Firm FS-1
Consolidated Income Statement FS-2
Consolidated Statement of Comprehensive Income FS-2
Sector Income Statement FS-3
Consolidated Balance Sheet FS-4
Sector Balance Sheet FS-5
Condensed Consolidated Statement of Cash Flows FS-6
Condensed Sector Statement of Cash Flows FS-7
Consolidated Statement of Equity FS-8
Notes to the Financial Statements FS-9
Schedule II — Valuation and Qualifying Accounts FSS-1
ii

output:
reasoning: The table of content is well structured in four parts, with the corresponding page numbers at the end of each line corresponding. There are also some subheadings that i will report. 
toc_elements:[
    {
    text: "Item 1 Business",
    label: "item01",
    page: "1"
    },
    ... other elements, followiing a fex examples of subheadings
    {
    text: "Overview",
    label: "subheading",
    page: "84"
    },
    {  
    text: "Automotive Sector",
    label: "subheading",
    page: "84"
    }, ...
    {
    text: "Consolidated Income Statement",
    label: "subheading",
    page: "FS-2"
    }, ...
]
"""


extract_toc_system = extract_toc_system + examples



In [6]:
class TocElement(BaseModel):
    text: str = Field(..., description="the text of the item heading or subheading")
    label: str = Field(..., description="the label of the heading or subheading")
    extracted_page_number: str = Field(..., description="the extracted page number of the item heading or subheading as string")


class AnswerTocExtractor(BaseModel):
    reasoning: str = Field(..., description="the reasoning behind your decision, including if you were able or not, and how you detected and correctly matches the headings/subheadings to their page numbers")
    toc_elements: List[TocElement] = Field(..., description="the list of elements extracted from the page")

parser_toc = PydanticOutputParser(pydantic_object=AnswerTocExtractor)

In [8]:
test_folder_path = 'outputs/train'

filenames_test = [f for f in os.listdir(test_folder_path) if f.startswith('tr')]
filenames_test = sorted(filenames_test)

for filename in filenames_test[88:]:
    print(f"Document ID: {filename}")

    df = pd.read_parquet(f'{test_folder_path}/{filename}/df_toc.parquet')

    # if df have no element with noise_type == 'toc', continue
    if df[df.noise_type == 'toc'].shape[0] == 0:
        print("No TOC elements detected in the document")
        print()
        continue

    toc_text = df[df['noise_type'] == 'toc']['md_text'].to_list()
    toc_text = '\n'.join(toc_text)

    sys_messge = (SystemMessage(content=extract_toc_system + "\n" + parser_toc.get_format_instructions()))
    human_message = HumanMessage(content=toc_text)
    messages = [sys_messge, human_message]

    llm = ChatOpenAI(model='gpt-4o', temperature=0)
    llm = llm.with_structured_output(AnswerTocExtractor)

    time.sleep(5)
    

    response = llm.invoke(messages)

    toc = response.toc_elements
    toc = [{'text':t.text, 'label':t.label, 'extracted_page_number':t.extracted_page_number} for t in toc]

    if toc:

        # Save the dict to a json file
        with open(f'{test_folder_path}/{filename}/toc.json', 'w') as file:
            json.dump(toc, file, indent=4)
        print("TOC elements extracted and saved")
    else:
        print("No TOC elements extracted")
    print()



Document ID: tr88_0001404912_0001047469-15-001418


KeyboardInterrupt: 